# Predicting NYC Food Deserts EDA

In [35]:
# import necessary packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import censusdata
from sodapy import Socrata
import warnings
warnings.filterwarnings('ignore')

# set directory
PROJECT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_DIR, 'images')
os.makedirs(IMAGES_PATH, exist_ok=True)

# function for saving figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Import data

### NYC Census Tracts Shapefile

In [36]:
# set client for request Socrata API
client = Socrata("data.cityofnewyork.us", app_token='2zRBf7lHUfSBGGX44PZy1XhmT')

# get shapefile from Socrata API
ct_shp = client.get("7igh-afai", limit=20000)

# convert to pandas GeoDataFrame
ct_df = pd.DataFrame.from_records(ct_shp)

### USDA data import

In [37]:
# import usa dataset
usda_df = pd.read_excel('data/DataDownload2015.xlsx', sheet_name='Food Access Research Atlas', header=0)

# create data dict df
usda_data_dict = pd.read_excel('data/DataDownload2015.xlsx', sheet_name='Variable Lookup') 

In [38]:
usda_df.head()

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,HUNVFlag,LowIncomeTracts,PovertyRate,MedianFamilyIncome,LA1and10,LAhalfand10,LA1and20,LATracts_half,LATracts1,LATracts10,LATracts20,LATractsVehicle_20,LAPOP1_10,LAPOP05_10,LAPOP1_20,LALOWI1_10,LALOWI05_10,LALOWI1_20,lapophalf,lapophalfshare,lalowihalf,lalowihalfshare,lakidshalf,lakidshalfshare,laseniorshalf,laseniorshalfshare,lawhitehalf,lawhitehalfshare,lablackhalf,lablackhalfshare,laasianhalf,laasianhalfshare,lanhopihalf,lanhopihalfshare,laaianhalf,laaianhalfshare,laomultirhalf,laomultirhalfshare,lahisphalf,lahisphalfshare,lahunvhalf,lahunvhalfshare,lasnaphalf,lasnaphalfshare,lapop1,lapop1share,lalowi1,lalowi1share,lakids1,lakids1share,laseniors1,laseniors1share,lawhite1,lawhite1share,lablack1,lablack1share,laasian1,laasian1share,lanhopi1,lanhopi1share,laaian1,laaian1share,laomultir1,laomultir1share,lahisp1,lahisp1share,lahunv1,lahunv1share,lasnap1,lasnap1share,lapop10,lapop10share,lalowi10,lalowi10share,lakids10,lakids10share,laseniors10,laseniors10share,lawhite10,lawhite10share,lablack10,lablack10share,laasian10,laasian10share,lanhopi10,lanhopi10share,laaian10,laaian10share,laomultir10,laomultir10share,lahisp10,lahisp10share,lahunv10,lahunv10share,lasnap10,lasnap10share,lapop20,lapop20share,lalowi20,lalowi20share,lakids20,lakids20share,laseniors20,laseniors20share,lawhite20,lawhite20share,lablack20,lablack20share,laasian20,laasian20share,lanhopi20,lanhopi20share,laaian20,laaian20share,laomultir20,laomultir20share,lahisp20,lahisp20share,lahunv20,lahunv20share,lasnap20,lasnap20share,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,0,0,0,0,0,10.0,74750,1,1,1,1,1,0,0,0,1357.480940,1732.225468,1357.480940,322.084612,411.690754,322.084612,1732.225468,0.905976,411.690754,0.215319,466.426430,0.243947,198.828220,0.103990,1482.881417,0.775566,184.319106,0.096401,12.718121,0.006652,0.000000,0.000000,13.969933,0.007306,38.336897,0.020051,39.695021,0.020761,21.556248,0.031106,101.877398,0.147009,1357.480940,0.709980,322.084612,0.168454,363.638381,0.190187,162.497246,0.084988,1161.802553,0.607637,147.487666,0.077138,11.015106,0.005761,0.000000,0.000000,9.996592,0.005228,27.179030,0.014215,29.715536,0.015542,9.772855,0.014102,79.546843,0.114786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,448,507,221,1622,217,14,0,14,45,44,26,112
1,1001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,0,0,0,0,0,18.2,51875,0,1,0,1,0,0,0,0,483.429683,1410.374828,483.429683,145.141418,475.306031,145.141418,1410.374828,0.649942,475.306031,0.219035,448.163512,0.206527,139.305390,0.064196,412.291349,0.189996,945.317796,0.435630,4.000000,0.001843,0.000000,0.000000,4.717582,0.002174,44.048101,0.020299,34.587909,0.015939,58.628965,0.078908,127.367745,0.171424,483.429683,0.222779,145.141418,0.066885,174.770470,0.080539,50.976822,0.023492,128.160995,0.059060,335.447721,0.154584,1.590927,0.000733,0.000000,0.000000,0.263511,0.000121,17.966531,0.008280,11.173590,0.005149,21.638337,0.029123,41.700964,0.056125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,763,606,214,888,1217,5,0,5,55,75,87,202
2,1001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,0,0,0,0,0,19.1,52905,1,1,1,1,1,0,0,0,1417.874893,2764.604126,1417.874893,696.636797,1350.176549,696.636797,2764.604126,0.819628,1350.176549,0.400290,744.891575,0.220839,346.203097,0.102640,2114.851375,0.626994,528.050101,0.156552,9.031083,0.002677,1.000000,0.000296,9.964775,0.002954,101.706798,0.030153,76.40861

In [39]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', -1)
usda_data_dict

,Field,LongName,Description
0,CensusTract,Census tract,Census tract number
1,State,State,State name
2,County,County,County name
3,Urban,Urban tract,Flag for urban tract
4,POP2010,"Population, tract total",Population count from 2010 census
5,OHU2010,"Housing units, total",Occupied housing unit count from 2010 census
6,GroupQuartersFlag,"Group quarters, tract with high share",Flag for tract where >=67%
7,NUMGQTRS,"Group quarters, tract population residing in, number",Count of tract population residing in group quarters
8,PCTGQTRS,"Group quarters, tract population residing in, share",Percent of tract population residing in group quarters
9,LILATracts_1And10,Low income and low access tract measured at 1 mile for urban areas and 10 miles for rural areas,Flag for food desert when considering low accessibilty at 1 and 10 miles


In [40]:
usda_data_dict.to_csv('data/usda_data_dict.csv')

### Data cleaning

Filter out data by state for New York only.

In [41]:
usda_df = usda_df.loc[usda_df['State'] == 'New York']


Filter out data for 5 boroughs

In [42]:
# create filter for boolean mask to isolate 5 boroughs
five_boroughs =  (usda_df['County'] == 'Bronx') | (usda_df['County'] == 'Kings') | (usda_df['County'] == 'New York') |\
        (usda_df['County'] == 'Queens') | (usda_df['County'] == 'Richmond')

# create usda nyc df
usda_df_nyc = usda_df.loc[five_boroughs]
usda_df_nyc.to_csv('data/usda_df_nyc.csv')

In [43]:
# look at resultant shape of df
usda_df_nyc.shape


(2165, 147)

There are 2165 rows (census tracts) in New York City 5 boroughs and 147 columns.

In [44]:
# get value counts for columsn
usda_df_nyc['County'].value_counts()

Kings       760
Queens      669
Bronx       339
New York    288
Richmond    109
Name: County, dtype: int64

Look at value counts for target column

In [45]:
# isolate target column to see classification
usda_df_nyc['LILATracts_halfAnd10'].value_counts()

0    2134
1    31  
Name: LILATracts_halfAnd10, dtype: int64

In [46]:
nyc_food_desert = usda_df['LILATracts_halfAnd10'] == 1
nyc_food_desert_df = usda_df.loc[nyc_food_desert]
nyc_food_desert_df.shape


(545, 147)

In [47]:
# look at dtypes of each column
usda_df_nyc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2165 entries, 43145 to 47000
Columns: 147 entries, CensusTract to TractSNAP
dtypes: float64(112), int64(33), object(2)
memory usage: 2.4+ MB


147 columns 112 floats, 33 ints and 2 objects

Drop unnecesary columns, only concerned with data of 1/2 mile access.

In [52]:
usda_df_nyc.drop(usda_df_nyc.iloc[:,57:135], inplace=True, axis=1)

In [56]:
usda_df_nyc.columns

Index(['CensusTract', 'State', 'County', 'Urban', 'POP2010', 'OHU2010',
       'GroupQuartersFlag', 'NUMGQTRS', 'PCTGQTRS', 'LILATracts_1And10',
       'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle',
       'HUNVFlag', 'LowIncomeTracts', 'PovertyRate', 'MedianFamilyIncome',
       'LA1and10', 'LAhalfand10', 'LA1and20', 'LATracts_half', 'LATracts1',
       'LATracts10', 'LATracts20', 'LATractsVehicle_20', 'LAPOP1_10',
       'LAPOP05_10', 'LAPOP1_20', 'LALOWI1_10', 'LALOWI05_10', 'LALOWI1_20',
       'lapophalf', 'lapophalfshare', 'lalowihalf', 'lalowihalfshare',
       'lakidshalf', 'lakidshalfshare', 'laseniorshalf', 'laseniorshalfshare',
       'lawhitehalf', 'lawhitehalfshare', 'lablackhalf', 'lablackhalfshare',
       'laasianhalf', 'laasianhalfshare', 'lanhopihalf', 'lanhopihalfshare',
       'laaianhalf', 'laaianhalfshare', 'laomultirhalf', 'laomultirhalfshare',
       'lahisphalf', 'lahisphalfshare', 'lahunvhalf', 'lahunvhalfshare',
       'lasnaphalf', 'lasna